<a href="https://colab.research.google.com/github/TrieuLe0801/craw_data_fedex/blob/master/crawl_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install pandas
!pip install joblib
!pip install bs4
!pip install fake_useragent

     |████████████████████████████████| 911kB 5.5MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/

In [ ]:
from selenium import webdriver
import time
import selenium.common.exceptions as exception
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import sys
import pandas as pd
import multiprocessing as mp
import threading
import itertools
import concurrent
from joblib import Parallel, delayed, parallel_backend
from queue import Queue
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
# threads= mp.cpu_count()
pool = mp.Pool(4)
q_task = Queue
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from pyvirtualdisplay import Display 
deliver_country=['United States','United Kingdom', 'France', 'Germany', 'Italy']
global dataframe 
dataframe = pd.DataFrame(columns=["Country_from","Country_to","Status","Shipment_number","Delivery"])
global result
result = []
global backup
backup = mp.Manager().list()
global list_pages
list_pages = mp.Manager().list()
lock = threading.RLock()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# chrome driver
def chrome_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--ignore-certificate-errors')
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--incognito')
  options.add_argument("--disable-gpu")
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.maximize_window
  return driver

In [ ]:
def save_csv(list_data,name):
  dataframe = pd.DataFrame(columns=["Country_from","Country_to","Status","Shipment_number","Delivery"])
  dataframe = pd.DataFrame(list_data, columns=dataframe.columns)
  print(dataframe)
  dataframe.to_csv('drive/My Drive/Colab Notebooks/fedex_data/{}.csv'.format(name), index=False,header=True)

In [ ]:
# def spliter(x):
#   return x.text.split("\n")

def filter(i,sub_list):
  item = i.text.split("\n")
  country_attribute = item[2].strip()
  if country_attribute.find(",") == -1:
    if country_attribute in deliever_country:
      if item.index("Shipment number") == 7:
        sub_list.append([item[0],item[2],item[4] +" "+item[5]+" "+ item[6],item[8],item[10],item[12]])
      else:
        sub_list.append([item[0],item[2],item[4],item[6],item[8],item[10]])
    else:
      pass
  else:
    if country_attribute.split(",")[1].strip() in deliever_country:
      if item.index("Shipment number") == 7:
        sub_list.append([item[0],item[2],item[4] +" "+item[5]+" "+ item[6],item[8],item[10],item[12]])
      else:
        sub_list.append([item[0],item[2],item[4],item[6],item[8],item[10]])
    else:
      pass

def filter_item(list_item):
  sub_list = []
  # pool.starmap(filter,[[i,sub_list] for i in list_item])
  # pool.close()
  for item in list_item:
    # item = i.text.split("\n")
    country_attribute = item[2].strip()
    if country_attribute.find(",") == -1:
      if country_attribute in deliever_country:
        if item.index("Shipment number") == 7:
          sub_list.append([item[0],item[2],item[4] +" "+item[5]+" "+ item[6],item[8],item[10],item[12]])
        else:
          sub_list.append([item[0],item[2],item[4],item[6],item[8],item[10]])
      else:
        continue
    else:
      if country_attribute.split(",")[1].strip() in deliever_country:
        if item.index("Shipment number") == 7:
          sub_list.append([item[0],item[2],item[4] +" "+item[5]+" "+ item[6],item[8],item[10],item[12]])
        else:
          sub_list.append([item[0],item[2],item[4],item[6],item[8],item[10]])
      else:
        continue
  return sub_list

# accept directly to list of item customer references
def crawl_data(customer_reference, driver_dict):
  list_item = []
  list_result = []
  driver = chrome_driver()
  try:
    driver.get('https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}'.format(customer_reference+1))
    driver_dict[customer_reference] = driver
    print('Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}'.format(customer_reference+1))
  except exception.InvalidSessionIdException:
    print("Cannot load this page")
    return list_result
  time.sleep(2)
  with lock:
    try:
      new_driver = driver_dict.get(customer_reference)
      accept_cookies = new_driver.find_element_by_xpath('//button[@class="cc-button cc-button--acceptAll"]')
      accept_cookies.click()
      print('Clicked accept all cookies')
    except NoSuchElementException:
      print("No need to click accept cookie button")

    # wait items appear (300 items)
    try:
      WebDriverWait(new_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h2[@class="__c-heading __c-heading--h2 __u-mb--none"]')))
      # print(new_driver.find_element_by_xpath('//h2[@class="__c-heading __c-heading--h2 __u-mb--none"]').text)
      # new_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      # time.sleep(2)
      # st = datetime.now()
      list_item = [item.text.split("\n") for item in new_driver.find_elements_by_xpath("//*[@class='__u-mb--xl']")]
      if len(list_item) > 0:
        print("Number of items in ref {}: {}".format(customer_reference+1,len(list_item)))
        list_result = filter_item(list_item)
        print("size of result of ref {}: {}".format(customer_reference+1,len(list_result)))
      else:
        print("Cannot load this ref {}".format(customer_reference + 1))
    except TimeoutException:
      print("Time out, cannot load this page {}".format(customer_reference))
    driver.quit()
  return list_result
  

In [ ]:
def crawl_data_bs4(cus_ref, driver_dict):
  list_item = []
  driver = chrome_driver()
  global save_time
  global pause_time
  try:
    driver.get('https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}'.format(cus_ref+1))
    driver_dict[cus_ref] = driver
    print('Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}'.format(cus_ref+1))
  except exception.InvalidSessionIdException:
    print("Cannot load this page")
    return list_item
  with lock:
    # Click cookies accepts
    try:
      new_driver = driver_dict.get(cus_ref)
      accept_cookies = new_driver.find_element_by_xpath('//button[@class="cc-button cc-button--acceptAll"]')
      accept_cookies.click()
      print('Clicked accept all cookies')
    except NoSuchElementException:
      print("No need to click accept cookie button")

    # load beautifulsoup
    try:
      WebDriverWait(new_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h2[@class="__c-heading __c-heading--h2 __u-mb--none"]')))
      html = new_driver.page_source
      soup = BeautifulSoup(html)
      new_driver.quit()
      list_item = [[item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[0].text.replace("\n",'').strip().replace(" ",""),
                          item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip().replace(" ",""),
                          item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip(),
                          item.find('div', class_='__c-shipment__reference-content').text.split('  ')[1],
                          item.find('span', class_='__u-hide--small-medium').text
                          ]
                    if item.find('span', class_='__u-hide--small-medium') != None
                    else [item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[0].text.replace("\n",'').strip().replace(" ",""),
                        item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip().replace(" ",""),
                        item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip(),
                        item.find('div', class_='__c-shipment__reference-content').text.split('  ')[1],
                        " "
                        ]
                    for item in\
                    list(filter(lambda item: item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip() != "Delivered"
                    and any([item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip() in deliver_country,
                             item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace('\n','').strip().find(",") != -1 and
                             item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace('\n','').strip().split(',')[1].strip() in deliver_country]),
                        soup.find_all('div',class_='__c-shipment')))
                  ]
      list_pages.remove(cus_ref)
      [backup.append(i) for i in list_item if i not in backup]
    except TimeoutException:
      print("Time out, cannot load this page {}".format(cus_ref+1))
      new_driver.quit()
    print("Size of items list of {}: {}".format(cus_ref+1,len(list_item)))
    timing = datetime.now()
    print("Timing:{}".format(timing - start))
  if (timing-save_time).total_seconds() >= 900:
    save_time = datetime.now()
    file_name = "fedex_data_backup"
    backup_list = [i for i in backup]
    save_csv(backup_list,file_name)
    print("Save csv to drive at {}".format(datetime.now()))
  pause_timing = datetime.now()
  if (pause_timing - pause_time).total_seconds() >= 1200:
    print("Wait 15 minutes to free space...")
    time.sleep(900)
    pause_time = datetime.now()
  return list_item

In [ ]:
def main(from_page, to_page):
  data = []
  global start
  start = datetime.now()
  global save_time
  save_time = start
  global pause_time
  pause_time = start
  # pause_time = start
  print("start at {}".format(start))
  list_pages.extend(range(from_page,to_page))
  while len(list_pages) != 0:
    [data.append(i) for i in list(itertools.chain.from_iterable(Parallel(n_jobs=5, backend='multiprocessing')(delayed(crawl_data_bs4)(j,{}) for j in list_pages)))
     if i not in data]
  print("Finish crawl at {}, after {}".format(datetime.now(),datetime.now()-start))
  return data
sub_list = []
sub_list = main(0,5)
result = sub_list
save_csv(result, "fedex_data")

start at 2020-11-18 09:13:12.109611
Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons=1
Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons=3
Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons=4
Clicked accept all cookies
Clicked accept all cookies
Clicked accept all cookies
Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons=2
Clicked accept all cookies


KeyboardInterrupt: ignored

In [ ]:
result = sub_list
new_result = []
[new_result.append(i) for i in result if i not in new_result]
# save_csv(result, "fedex_data_{}".format(datetime.now()))
print(len(result))
# for i in result:
#   if i not in new_result:
#     new_result.append(i)
print(len(new_result))
dataframe = pd.DataFrame(new_result,columns=dataframe.columns)

1216
1056


In [ ]:
driver_dict = {}
start = datetime.now()
with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(crawl_data, i, {}) for i in range(100)]

for future in concurrent.futures.as_completed(futures):
    raw_record.extend(future.result())
    print(len(raw_record))
print("Finish crawl at {}, after {}".format(datetime.now(),datetime.now()-start))

In [ ]:
import argparse
from selenium import webdriver
import time
import selenium.common.exceptions as exception
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import sys
import pandas as pd
import multiprocessing as mp
import threading
import itertools
import concurrent
from joblib import Parallel, delayed, parallel_backend
from queue import Queue
from bs4 import BeautifulSoup
import random
from fake_useragent import UserAgent
sys.path.insert(0,'chromedriver')
deliver_country=['United States','United Kingdom', 'France', 'Germany', 'Italy']
dataframe = pd.DataFrame(columns=["Country_from","Country_to","Status","Shipment_number","Delivery"])
lock = threading.RLock()
start = None
save_time = None
pause_time = None

def updateSaveTime():
	global save_time
	save_time = datetime.now()

def updatePauseTime():
	global pause_time
	pause_time = datetime.now()

# chrome driver
def chrome_driver():
	options = webdriver.ChromeOptions()
	options.add_argument('--user-agent={}'.format(UserAgent().random))
	options.add_experimental_option("excludeSwitches",["ignore-certificate-errors"])
	options.add_argument('--proxy-server={}'.format("https://179.48.251.236:32"))
	options.add_argument('ignore-certificate-errors')
	# options.add_argument('headless')
	options.add_argument('no-sandbox')
	options.add_argument('disable-dev-shm-usage')
	options.add_argument('incognito')
	options.add_argument('disable-gpu')
	options.add_argument('allow-insecure-localhost')
  driver = webdriver.Chrome('chromedriver',options=options)
	driver.maximize_window
	return driver

def save_csv(list_data,name):
	dataframe = pd.DataFrame(columns=["Country_from","Country_to","Status","Shipment_number","Delivery"])
	dataframe = pd.DataFrame(list_data, columns=dataframe.columns)
	print(dataframe)
	dataframe.to_csv('{}.csv'.format(name), index=False,header=True)

def login_petco():
  driver = chrome_driver()
  driver.implicitly_wait(60)

def crawl_data_bs4(cus_ref, driver_dict,list_pages,backup,start,list_time_control):
	list_item = []
	driver = chrome_driver()
	driver.implicitly_wait(60)
	try:
		try:
			driver.get('https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}'.format(cus_ref+1))
			driver_dict[cus_ref] = driver
			print("Loading: https://www.tnt.com/express/en_gb/site/shipping-tools/tracking.html?searchType=ref&cons={}".format(cus_ref+1))
		except exception.WebDriverException:
			print("Time out, cannot load this page")
			driver.close()
			driver.quit()
			return list_item
	except exception.InvalidSessionIdException:
		print("cannot load this page")
		driver.close()
		driver.quit()
		return list_item
	with lock:
	# Click cookies accepts
		try:
			new_driver = driver_dict.get(cus_ref)
			accept_cookies = new_driver.find_element_by_xpath('//button[@class="cc-button cc-button--acceptAll"]')
			accept_cookies.click()
			print('Clicked accept all cookies')
		except NoSuchElementException:
			print("No need to click accept cookie button")
		# load beautifulsoup
		try:
			WebDriverWait(new_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h2[@class="__c-heading __c-heading--h2 __u-mb--none"]')))
			html = new_driver.page_source
			soup = BeautifulSoup(html,"html.parser")
			new_driver.close()
			new_driver.quit()
			list_item = [[item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[0].text.replace("\n",'').strip().replace("  "," "),
								item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip().replace("  "," "),
								item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip(),
								item.find('div', class_='__c-shipment__reference-content').text.split('  ')[1],
								item.find('span', class_='__u-hide--small-medium').text
								]
						if item.find('span', class_='__u-hide--small-medium') != None
						else [item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[0].text.replace("\n",'').strip().replace("  "," "),
							item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip().replace("  "," "),
							item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip(),
							item.find('div', class_='__c-shipment__reference-content').text.split('  ')[1],
							" "
							]
						for item in\
						list(filter(lambda item: item.find('div',class_='__c-shipment-status-tnt__summary').text.replace("\n",'').strip() != "Delivered"
						and any([item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace("\n",'').strip() in deliver_country,
									item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace('\n','').strip().find(",") != -1 and
									item.find_all('div',class_='__c-heading __c-heading--h4 __c-heading--bold __u-mb--none')[1].text.replace('\n','').strip().split(',')[1].strip() in deliver_country]),
							soup.find_all('div',class_='__c-shipment')))
						]
			list_pages.remove(cus_ref)
			[backup.append(i) for i in list_item if i not in backup]
		except TimeoutException:
			print("Time out, cannot load this page {}".format(cus_ref+1))
			time.sleep(1)
			new_driver.close()
			new_driver.quit()
		print("Size of items list of {}: {}".format(cus_ref+1,len(list_item)))
		print("Timing:{}".format(datetime.now()	- start))
	if (datetime.now()-list_time_control[0]).total_seconds() >= 900:
		file_name = "fedex_data"
		backup_list = [i for i in backup]
		save_csv(backup_list,file_name)
		print("Save csv to drive at {}".format(datetime.now()))
		list_time_control[0] = datetime.now()
	if (datetime.now() - list_time_control[1]).total_seconds() >= 3600:
		file_name = "fedex_data_backup"
		backup_list = [i for i in backup]
		save_csv(backup_list,file_name)
		print("Wait 5 minutes to free space...")
		time.sleep(300)
		list_time_control[1] = datetime.now()
	return list_item

def main(from_page=0, to_page=10000):
	data = []
	start = datetime.now()
	save_time = datetime.now()
	pause_time = datetime.now()
	backup = mp.Manager().list()
	list_pages = mp.Manager().list()
	list_time_control = mp.Manager().list()
	list_time_control.append(save_time)
	list_time_control.append(pause_time)
	#print("start at {}".format(start))
	list_pages.extend(range(from_page,to_page))
	while len(list_pages) != 0:
		print("start at {}".format(start))
		print("Leftover :{}".format(len(list_pages)))
		[data.append(i) for i in list(itertools.chain.from_iterable(Parallel(n_jobs=2, backend='multiprocessing')(delayed(crawl_data_bs4)(j,{},list_pages,backup,start,list_time_control) 
			for j in list_pages)))
			if i not in data]
	print("Finish crawl at {}, after {}".format(datetime.now(),datetime.now()-start))
	save_csv(data, "fedex_data")

if __name__=="__main__":
  login_petco()
	# parser = argparse.ArgumentParser()
	# parser.add_argument('--from_page', action="store")
	# parser.add_argument('--to_page',action="store")
	# args = parser.parse_args()
	# while(True):
	# 	if args.from_page != None and args.to_page != None:
	# 		main(int(args.from_page),int(args.to_page))
	# 	else:
	# 		main()
	# 	print("Done at {}. Wait 10 minutes before start new".format(datetime.now()))
	# 	time.sleep(600)

IndentationError: ignored

In [ ]:
import argparse
from selenium import webdriver
import time
import selenium.common.exceptions as exception
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import sys
import pandas as pd
import multiprocessing as mp
import threading
import itertools
import concurrent
from joblib import Parallel, delayed, parallel_backend
from queue import Queue
from bs4 import BeautifulSoup
import random
from fake_useragent import UserAgent
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# chrome driver
def chrome_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--ignore-certificate-errors')
  # options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--incognito')
  options.add_argument("--disable-gpu")
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.maximize_window
  return driver

def login_game():
  driver = chrome_driver()
  driver.implicitly_wait(60)
  try:
    try:
      driver.get('http://202.158.245.65/')
			# driver_dict[cus_ref] = driver
      print("Loading: http://202.158.245.65/")
    except exception.WebDriverException:
      print("Time out, cannot load this page")
      driver.close()
      driver.quit()
      return list_item
  except exception.InvalidSessionIdException:
    print("cannot load this page")
    driver.close()
    driver.quit()
    return list_item
  try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//input[@id="imeDispatcherInput"]')))
  except TimeoutException:
    print("Time out, cannot load this page ")
    time.sleep(1)
    driver.close()
    driver.quit()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
login_game()

WebDriverException: ignored